# Integrated Data

=========================== Step1: Merge 'lyrics.csv'(df1) and 'SPARQL_1.csv'(df2) into 'df' according to 'artistName'========================

这里的思路是按照一两个主键，先把两个数据集merge起来，然后剩下的数据集依次按照一两个主键merge到之前merge起来的数据集里。  
每个主键要进行normalize。为了让处理完后的结果差不多，每一步normalize主键的过程大同小异。  
大致是去除主键里的特殊符号，- " 首尾空格等。进行重命名便于标识。artistName那里，如果有一个以上的人名，就用&连接。如：Earth Wind & Fire  
每merge进一个dataset，就根据主键去重一下。

In [119]:
import pandas as pd
import numpy as np
import re
import json
from pandas.core.frame import DataFrame
import  xml.dom.minidom
path = 'C:\\Users\\D074110\\Desktop\\Study\\课\\Web Data Integration\\WDI_TP\\Data collection\\final datasets\\'

In [120]:
# Loading 'lyrics.csv'. What does the original 'song' looks like in 'lyrics.csv'
f1 = open(path + 'lyrics.csv',encoding="utf8")
df1=pd.read_csv(f1,sep=',')
#df1['song'] 
#All the words are lowercase. But there are some capitals in another dataset. So we should normalize

In [121]:
#Normalizing songName in 'lyrics.csv'
df1['song']=df1['song'].str.replace('-',' ') # Remove - inside of songName
df1['song']=df1['song'].str.title()# Every word in the songName beginning with capital
df1=df1.rename(columns={'year':'songReleaseYear'}) #Reaname 'song' with 'songName'
df1=df1.rename(columns={'genre':'songGenre'})
df1=df1.rename(columns={'artist':'artistName'})
df1=df1.rename(columns={'song':'songName'})
print(df1.head())
print(df1.shape)
#df1['songName'] #Check songName of 'lyrics.csv'

   index         songName  songYear       artistName songGenre  \
0      0        Ego Remix      2009  beyonce-knowles       Pop   
1      1     Then Tell Me      2009  beyonce-knowles       Pop   
2      2          Honesty      2009  beyonce-knowles       Pop   
3      3  You Are My Rock      2009  beyonce-knowles       Pop   
4      4    Black Culture      2009  beyonce-knowles       Pop   

                                              lyrics  
0  Oh baby, how you doing?\nYou know I'm gonna cu...  
1  playin' everything so easy,\nit's like you see...  
2  If you search\nFor tenderness\nIt isn't hard t...  
3  Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote...  
4  Party the people, the people the party it's po...  
(362237, 6)


In [122]:
# Loading 'SPARQL_1.csv'. What does the original 'song' looks like in 'SPARQL_1.csv'
f2 = open(path + 'SPARQL_1.csv',encoding="utf8")
df2=pd.read_csv(f2,sep='" , "', error_bad_lines=False,warn_bad_lines=False)
#df2['songName'] 

C:\Users\D074110\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [123]:
#Normalizing songName in SPARQL_1.csv'
#Remove ""
df2 = df2.rename(columns=lambda x: x.replace("'","").replace('"','')).replace(" ","") 
df2['Artistname']=df2['Artistname'].str.replace('"','')
df2['songName']=df2['songName'].str.replace('"','')
df2['songName']=df2['songName'].str.replace(' ,  ,  ,  ,','')#Remove ,  ,  ,  ,
df2['songName']=df2['songName'].str.title()# Every word in the songName beginning with capital
df2=df2.rename(columns={'Artistname':'artistName'})
df1=df1.rename(columns={'artist':'artistName'})
print(df2.head())
print(df2.shape)

       artistName                               songName songRecordDate  \
0  Charlie Parker                        Billie'S Bounce           None   
1           Cream             Doing That Scrapyard Thing           None   
2  Dannii Minogue  Come And Get It (Dannii Minogue Song)           None   
3      Gary Numan                           Metal (Song)           None   
4         Genesis                         Firth Of Fifth           None   

  songLyrics   v ,  
0       None  None  
1       None  None  
2       None  None  
3       None  None  
4       None  None  
(5218, 5)


In [124]:
#Remove () by using regular expression
songName = df2['songName']
songName = ','.join(str(v) for v in songName)  # transfer data into string. Because re.sub() expected string or bytes-like object
pattern=re.compile(r'\((.*?)\)')
songName = pattern.sub('',songName)
songName = songName.strip() 
songName = songName.split(',') #transfer data into list
songName = [line.strip() for line in songName]  #remove skip at the begging and end of string
#songName #It's list now

In [125]:
#append data from list to dataframe
df2['songName'] = pd.Series(songName)
df2['songName']=df2['songName'].str.replace('-',' ') 
print(df2.head())
print(df2.shape)
#df2['songName']

       artistName                    songName songRecordDate songLyrics   v ,
0  Charlie Parker             Billie'S Bounce           None       None  None
1           Cream  Doing That Scrapyard Thing           None       None  None
2  Dannii Minogue             Come And Get It           None       None  None
3      Gary Numan                       Metal           None       None  None
4         Genesis              Firth Of Fifth           None       None  None
(5218, 5)


In [126]:
df1['artistName'] =df1['artistName'].str.replace(' / ',' ') # Remove / inside of Artist
df1['artistName'] =df1['artistName'].str.replace('-',' ') # Remove / inside of Artist
df1['artistName'] =df1['artistName'].str.replace(', ',' ') # Remove , inside of Artist
df1['artistName'] =df1['artistName'].str.title()# Every word in the songName beginning with capital
#df1['artistName'] 

In [127]:
df2['artistName'] =df2['artistName'].str.replace(' / ',' ') # Remove / inside of Artist
df2['artistName'] =df2['artistName'].str.replace('-',' ') # Remove / inside of Artist
df2['artistName'] =df2['artistName'].str.replace(', ',' ') # Remove , inside of Artist
df2['artistName'] =df2['artistName'].str.title()# Every word in the songName beginning with capital
#df2['artistName']

In [128]:
#merge two datasets according to 'songName' into a new dataset
df = pd.merge(df1, df2, on = ['songName','artistName'],how = 'outer')
#Remove duplicates value accoeding to 'songName' for each dataset 
df = df.drop_duplicates(['songName','artistName'],keep='first') 
#drop some useless columns
df = df.drop(['songRecordDate','index','v ,'], 1, inplace=False)
print(df.head())
print(df.shape)

#删除列的两种方式
#df.drop(columns=['a','b'])
#df.drop(['a','b'],axis=1)

          songName  songYear       artistName songGenre  \
0        Ego Remix    2009.0  Beyonce Knowles       Pop   
1     Then Tell Me    2009.0  Beyonce Knowles       Pop   
2          Honesty    2009.0  Beyonce Knowles       Pop   
3  You Are My Rock    2009.0  Beyonce Knowles       Pop   
4    Black Culture    2009.0  Beyonce Knowles       Pop   

                                              lyrics songLyrics  
0  Oh baby, how you doing?\nYou know I'm gonna cu...        NaN  
1  playin' everything so easy,\nit's like you see...        NaN  
2  If you search\nFor tenderness\nIt isn't hard t...        NaN  
3  Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote...        NaN  
4  Party the people, the people the party it's po...        NaN  
(366734, 6)


In [129]:
#print(df['artist'].notnull()) 显示某列中非空值
df.shape[0] - df.count()# 每列缺失值的个数

songName           2
songYear        5040
artistName         0
songGenre       5040
lyrics        100687
songLyrics    366718
dtype: int64

============================== Step2: Merge 'albumlist.csv'(df3) and df according to 'artistName'===========================

In [130]:
# Loading 'albumlist.csv'. 
f3 = open(path + 'albumlist.csv',encoding="ISO-8859-1")
df3=pd.read_csv(f3,sep=',')
df3=df3.rename(columns={'Year':'albumYear'})
df3=df3.rename(columns={'Genre':'albumGenre'})
df3=df3.rename(columns={'Subgenre':'albumSubgenre'})
df3=df3.rename(columns={'Album':'albumName'})
print(df3.head())
print(df3.shape)

   Number  albumYear                              albumName          Artist  \
0       1       1967  Sgt. Pepper's Lonely Hearts Club Band     The Beatles   
1       2       1966                             Pet Sounds  The Beach Boys   
2       3       1966                               Revolver     The Beatles   
3       4       1965                   Highway 61 Revisited       Bob Dylan   
4       5       1965                            Rubber Soul     The Beatles   

  albumGenre                  albumSubgenre  
0       Rock  Rock & Roll, Psychedelic Rock  
1       Rock     Pop Rock, Psychedelic Rock  
2       Rock     Psychedelic Rock, Pop Rock  
3       Rock          Folk Rock, Blues Rock  
4  Rock, Pop                       Pop Rock  
(500, 6)


In [131]:
#df3['Artist']=df3['Artist'].str.replace(' & ',' ') # Remove & inside of Artist
df3['Artist']=df3['Artist'].str.replace(' / ',' ') # Remove / inside of Artist
df3['Artist']=df3['Artist'].str.replace(', ',' ') # Remove , inside of Artist
df3['Artist']=df3['Artist'].str.title()# Every word in the songName beginning with capital
df3=df3.rename(columns={'Artist':'artistName'}) #Reaname 'Artist' with 'artistName'
df3['artistName'] 

0                           The Beatles
1                        The Beach Boys
2                           The Beatles
3                             Bob Dylan
4                           The Beatles
5                           Marvin Gaye
6                    The Rolling Stones
7                             The Clash
8                             Bob Dylan
9                           The Beatles
10                        Elvis Presley
11                          Miles Davis
12               The Velvet Underground
13                          The Beatles
14          The Jimi Hendrix Experience
15                            Bob Dylan
16                              Nirvana
17                    Bruce Springsteen
18                         Van Morrison
19                      Michael Jackson
20                          Chuck Berry
21                       Robert Johnson
22         John Lennon Plastic Ono Band
23                        Stevie Wonder
24                          James Brown


In [132]:
df = pd.merge(df, df3, on = ['artistName'],how = 'outer')
#drop some useless columns
df = df.drop(['Number','albumSubgenre'], 1, inplace=False)
print(df.head())
print(df.shape)

          songName  songYear       artistName songGenre  \
0        Ego Remix    2009.0  Beyonce Knowles       Pop   
1     Then Tell Me    2009.0  Beyonce Knowles       Pop   
2          Honesty    2009.0  Beyonce Knowles       Pop   
3  You Are My Rock    2009.0  Beyonce Knowles       Pop   
4    Black Culture    2009.0  Beyonce Knowles       Pop   

                                              lyrics songLyrics albumName  
0  Oh baby, how you doing?\nYou know I'm gonna cu...        NaN       NaN  
1  playin' everything so easy,\nit's like you see...        NaN       NaN  
2  If you search\nFor tenderness\nIt isn't hard t...        NaN       NaN  
3  Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote...        NaN       NaN  
4  Party the people, the people the party it's po...        NaN       NaN  
(395126, 7)


In [133]:
df.shape[0] - df.count()# 每列缺失值的个数

songName         141
songYear       10818
artistName         0
songGenre      10818
lyrics        107596
songLyrics    395078
albumName     349935
dtype: int64

============================== Step3: Merge 'SPARQL_3.json'(df4) and df according to 'artistName'===========================

这个数据集里"BandName"和"albumsNumber"都没有query出来。只有artisNationality和artisName

In [209]:
import json
import requests

In [210]:
#读取这个目录的JSON文件并解码成python数据
with open(path + 'SPARQL_31.json','r') as load_f:
    df4 = json.load(load_f) 
    #print(df4)
df4 = df4['results']#在这里是字典，字典才有key
#print(df4.keys()) #dict_keys(['distinct', 'ordered', 'bindings'])
df4 = df4['bindings'] #从这里开始就是list了
print(df4[1])

{'artistName': {'type': 'literal', 'xml:lang': 'en', 'value': 'John Matarazzo'}, 'artisNationality': {'type': 'literal', 'xml:lang': 'en', 'value': 'United States'}}


In [212]:
#先把list转成dataframe
from pandas.core.frame import DataFrame
df4=DataFrame(df4)
print(type(df4))
print(df4.head())
print(df4.shape)

<class 'pandas.core.frame.DataFrame'>
                                    artisNationality  \
0  {'type': 'literal', 'xml:lang': 'en', 'value':...   
1  {'type': 'literal', 'xml:lang': 'en', 'value':...   
2  {'type': 'literal', 'xml:lang': 'en', 'value':...   
3  {'type': 'literal', 'xml:lang': 'en', 'value':...   
4  {'type': 'literal', 'xml:lang': 'en', 'value':...   

                                          artistName  
0  {'type': 'literal', 'xml:lang': 'en', 'value':...  
1  {'type': 'literal', 'xml:lang': 'en', 'value':...  
2  {'type': 'literal', 'xml:lang': 'en', 'value':...  
3  {'type': 'literal', 'xml:lang': 'en', 'value':...  
4  {'type': 'literal', 'xml:lang': 'en', 'value':...  
(1604, 2)


In [213]:
#提取artistName的值。趁着是str赶紧normalize
for i in range(len(df4['artistName'])): #遍历dataframe中的一列
    #print(type(df4['artistName'][i])) <class 'dict'> 每一行都是dictionary
    an = df4['artistName'][i]
    artistName = an.get('value')#<class 'str'>现在每个都是str的形式，很适normalizing里面的值
    pattern=re.compile(r'\((.*?)\)')
    artistName = pattern.sub('',artistName)
    artistName = artistName.strip() 
    artistName = artistName.replace('-',' ')
    artistName = artistName.title()
    #print(artistName)
    df4 = df4.set_value(i, 'artistName', artistName)
    #set_value: Put single value at passed column and index
print(df4.head())
print(df4.shape)

                                    artisNationality         artistName
0  {'type': 'literal', 'xml:lang': 'en', 'value':...   Blanche Calloway
1  {'type': 'literal', 'xml:lang': 'en', 'value':...     John Matarazzo
2  {'type': 'literal', 'xml:lang': 'en', 'value':...      Richard Stone
3  {'type': 'literal', 'xml:lang': 'en', 'value':...  Bianca Maria Meda
4  {'type': 'literal', 'xml:lang': 'en', 'value':...      Girolamo Abos
(1604, 2)


C:\Users\D074110\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':


In [215]:
#同样的操作提取artisNationality
for i in range(len(df4['artisNationality'])): #遍历dataframe中的一列
    #print(type(df4['artistName'][i])) <class 'dict'> 每一行都是dictionary
    an = df4['artisNationality'][i]
    artisNationality = an.get('value')#<class 'str'>现在每个都是str的形式，很适normalizing里面的值
    pattern=re.compile(r'\((.*?)\)')
    artisNationality = pattern.sub('',artisNationality)
    artisNationality = artisNationality.strip() 
    artisNationality = artisNationality.replace('-',' ')
    artisNationality = artisNationality.title()
    #print(artistName)
    df4 = df4.set_value(i, 'artisNationality', artisNationality)
    #set_value: Put single value at passed column and index
print(df4.head())
print(df4.shape)

  artisNationality         artistName
0    United States   Blanche Calloway
1    United States     John Matarazzo
2    United States      Richard Stone
3            Italy  Bianca Maria Meda
4            Italy      Girolamo Abos
(1604, 2)


C:\Users\D074110\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':


In [216]:
df = pd.merge(df, df4, on = ['artistName'],how = 'outer')
#drop some useless columns
#df = df.drop(['Number','albumSubgenre','albumGenre','albumYear'], 1, inplace=False)
print(df.head())
print(df.shape)

          songName  songYear       artistName songGenre  \
0        Ego Remix    2009.0  Beyonce Knowles       Pop   
1     Then Tell Me    2009.0  Beyonce Knowles       Pop   
2          Honesty    2009.0  Beyonce Knowles       Pop   
3  You Are My Rock    2009.0  Beyonce Knowles       Pop   
4    Black Culture    2009.0  Beyonce Knowles       Pop   

                                              lyrics songLyrics albumName  \
0  Oh baby, how you doing?\nYou know I'm gonna cu...        NaN       NaN   
1  playin' everything so easy,\nit's like you see...        NaN       NaN   
2  If you search\nFor tenderness\nIt isn't hard t...        NaN       NaN   
3  Oh oh oh I, oh oh oh I\n[Verse 1:]\nIf I wrote...        NaN       NaN   
4  Party the people, the people the party it's po...        NaN       NaN   

  artisNationality  
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4              NaN  
(396728, 8)


In [217]:
df.shape[0] - df.count()# 每列缺失值的个数

songName              1708
songYear             12385
artistName               0
songGenre            12385
lyrics              109164
songLyrics          396680
albumName           351537
artisNationality    394652
dtype: int64

============================== Step4: Merge 'SPARQL_4.xml'(df5) and df according to 'BandMembers'===========================  
因为看了下基本只有只有一个成员

In [218]:
import  xml.dom.minidom
df5 = xml.dom.minidom.parse(path + 'SPARQL_4.xml')
print(df5)